In [75]:
import pandas as pd

INPUT_CSV = "/kaggle/input/grand-xray-slam-division-a/train1.csv"
OUTPUT_CSV = "/kaggle/working/Filtered"

In [76]:
df = pd.read_csv(INPUT_CSV)
df.head(5)

,Image_name,Patient_ID,Study,Sex,Age,ViewCategory,ViewPosition,Atelectasis,Cardiomegaly,Consolidation,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,00000001_001_001.jpg,1,1,Female,68.0,Frontal,AP,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,00000009_001_001.jpg,9,1,Male,76.0,Frontal,PA,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,00000009_001_002.jpg,9,1,Male,76.0,Lateral,Lateral,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,00000011_013_001.jpg,11,13,Female,22.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,00000011_013_002.jpg,11,13,Female,22.0,Lateral,Lateral,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [77]:
#All diseases in the dataset
ALL_DISEASE_COLUMNS = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Lung Opacity",
    "Pleural Effusion",
    "Pneumonia",
    "Pneumothorax",
    "Support Devices",
    "No Finding"
]

In [78]:
TARGET_CLASSES = [
    "Pneumothorax", #Hardest to detect
    "Cardiomegaly", #Easy to detect
    "Lung Opacity", #Most Common in Real World
    "Pleural Effusion", #Very Common in Real World
    "Support Devices" #Easy to Detect
]

In [79]:
META_DATA = [
    "Image_name",
    "Sex",
    "ViewCategory",
    "ViewPosition",
]

In [80]:
#FILTER: Frontal Only
df = df[
    (df["ViewCategory"] == "Frontal") &
    (df["ViewPosition"].isin(["AP", "PA"]))
].copy()

#FILTER: Target Diseases
df["target_positive_count"] = df[TARGET_CLASSES].sum(axis=1)
df = df[df["target_positive_count"] == 1].copy()

NON_TARGET_CLASSES = list(set(ALL_DISEASE_COLUMNS) - set(TARGET_CLASSES))
df["non_target_positive_count"] = df[NON_TARGET_CLASSES].sum(axis=1)
df = df[df["non_target_positive_count"] == 0].copy()

#Assigning Class Lables
df["label"] = df[TARGET_CLASSES].idxmax(axis=1)

#Final Cleanup
df = df[META_DATA + TARGET_CLASSES]
df.head(5)

,Image_name,Sex,ViewCategory,ViewPosition,Pneumothorax,Cardiomegaly,Lung Opacity,Pleural Effusion,Support Devices
0,00000001_001_001.jpg,Female,Frontal,AP,0,0,0,0,1
1,00000009_001_001.jpg,Male,Frontal,PA,0,1,0,0,0
19,00000017_001_001.jpg,Female,Frontal,AP,0,0,0,0,1
21,00000018_001_001.jpg,Male,Frontal,PA,0,0,0,1,0
44,00000030_001_001.jpg,Male,Frontal,AP,0,0,0,0,1


In [81]:
#Images Per Class
df["label"] = df[TARGET_CLASSES].idxmax(axis=1)
df["label"].value_counts()

label
Lung Opacity        4543
Support Devices     3527
Pleural Effusion    2131
Pneumothorax        1194
Cardiomegaly        1059
Name: count, dtype: int64

In [82]:
print("Total Images = ",len(df))
df.to_csv(OUTPUT_CSV, index=False)
print("The file is saved!!")

Total Images =  12454
The file is saved!!
